# OpenAI APIs - Embedding

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/guides/embeddings).

This tutorial covers the embedding APIs for embedding models, such as  
- [intfloat/e5-mistral-7b-instruct](https://huggingface.co/intfloat/e5-mistral-7b-instruct)  
- [Alibaba-NLP/gte-Qwen2-7B-instruct](https://huggingface.co/Alibaba-NLP/gte-Qwen2-7B-instruct)  


## Launch A Server

Launch the server in your terminal and wait for it to initialize. Remember to add `--is-embedding` to the command.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

embedding_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

/data/chayenne/.python/sglang/lib/python3.11/site-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
INFO 02-19 22:09:02 __init__.py:190] Automatically detected platform cuda.


2025-02-19 22:09:04,071 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


[2025-02-19 22:09:04] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, is_embedding=True, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=30027, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=109574069, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=Fals

/data/chayenne/.python/sglang/lib/python3.11/site-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
/data/chayenne/.python/sglang/lib/python3.11/site-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
INFO 02-19 22:09:08 __init__.py:190] Automatically detected platform cuda.
INFO 02-19 22:09:08 __init__.py:190] Automatically detected platform cuda.


2025-02-19 22:09:10,353 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
2025-02-19 22:09:10,415 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


...

[2025-02-19 22:09:25 TP0] KV Cache is allocated. K size: 4.45 GB, V size: 4.45 GB.
[2025-02-19 22:09:25 TP0] Memory pool end. avail mem=8.38 GB


[2025-02-19 22:09:26 TP0] max_total_num_tokens=166528, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072
[2025-02-19 22:09:26] INFO:     Started server process [1809904]
[2025-02-19 22:09:26] INFO:     Waiting for application startup.
[2025-02-19 22:09:26] INFO:     Application startup complete.
[2025-02-19 22:09:26] INFO:     Uvicorn running on http://0.0.0.0:30027 (Press CTRL+C to quit)
[2025-02-19 22:09:26] INFO:     127.0.0.1:60568 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-19 22:09:27] INFO:     127.0.0.1:60576 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-19 22:09:27 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-02-19 22:09:28,133 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-02-19 22:09:28,159 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
[2025-02-19 22:09:28] INFO:     127.0.0.1:60586 - "POST /encode HTTP/1.1" 200 OK
[2025-02-19 22:09:28] The server is fired up and ready to roll!




                    NOTE: Typically, the server runs in a separate terminal.
                    In this notebook, we run the server and notebook code together, so their outputs are combined.
                    To improve clarity, the server logs are displayed in the original black color, while the notebook outputs are highlighted in blue.
                    We are running those notebooks in a CI parallel environment, so the throughput is not representative of the actual performance.
                    


## Using cURL

In [2]:
import subprocess, json

text = "Once upon a time"

curl_text = f"""curl -s http://localhost:{port}/v1/embeddings \
  -d '{{"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "input": "{text}"}}'"""

text_embedding = json.loads(subprocess.check_output(curl_text, shell=True))["data"][0][
    "embedding"
]

print_highlight(f"Text embedding (first 10): {text_embedding[:10]}")

[2025-02-19 22:09:31 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-19 22:09:31] INFO:     127.0.0.1:34132 - "POST /v1/embeddings HTTP/1.1" 200 OK
Text embedding (first 10): [0.00830841064453125, 0.000698089599609375, -0.008087158203125, -0.0006761550903320312, 0.01438140869140625, -0.00899505615234375, 0.012359619140625, 0.00208282470703125, 0.00620269775390625, -0.003021240234375]


## Using Python Requests

In [3]:
import requests

text = "Once upon a time"

response = requests.post(
    f"http://localhost:{port}/v1/embeddings",
    json={"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "input": text},
)

text_embedding = response.json()["data"][0]["embedding"]

print_highlight(f"Text embedding (first 10): {text_embedding[:10]}")

[2025-02-19 22:09:31 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 3, cache hit rate: 21.43%, token usage: 0.00, #running-req: 0, #queue-req: 0
2025-02-19 22:09:31,602 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-02-19 22:09:31,615 - INFO - flashinfer.jit: Finished loading JIT ops: cascade
[2025-02-19 22:09:31] INFO:     127.0.0.1:34138 - "POST /v1/embeddings HTTP/1.1" 200 OK
Text embedding (first 10): [0.0083160400390625, 0.0006837844848632812, -0.00809478759765625, -0.0006804466247558594, 0.0143585205078125, -0.0089874267578125, 0.0123748779296875, 0.002101898193359375, 0.006221771240234375, -0.003025054931640625]


## Using OpenAI Python Client

In [4]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

# Text embedding example
response = client.embeddings.create(
    model="Alibaba-NLP/gte-Qwen2-7B-instruct",
    input=text,
)

embedding = response.data[0].embedding[:10]
print_highlight(f"Text embedding (first 10): {embedding}")

[2025-02-19 22:09:31 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 3, cache hit rate: 33.33%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-19 22:09:31] INFO:     127.0.0.1:34150 - "POST /v1/embeddings HTTP/1.1" 200 OK
Text embedding (first 10): [0.0083160400390625, 0.0006837844848632812, -0.00809478759765625, -0.0006804466247558594, 0.0143585205078125, -0.0089874267578125, 0.0123748779296875, 0.002101898193359375, 0.006221771240234375, -0.003025054931640625]


## Using Input IDs

SGLang also supports `input_ids` as input to get the embedding.

In [5]:
import json
import os
from transformers import AutoTokenizer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

tokenizer = AutoTokenizer.from_pretrained("Alibaba-NLP/gte-Qwen2-7B-instruct")
input_ids = tokenizer.encode(text)

curl_ids = f"""curl -s http://localhost:{port}/v1/embeddings \
  -d '{{"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "input": {json.dumps(input_ids)}}}'"""

input_ids_embedding = json.loads(subprocess.check_output(curl_ids, shell=True))["data"][
    0
]["embedding"]

print_highlight(f"Input IDs embedding (first 10): {input_ids_embedding[:10]}")

[2025-02-19 22:09:32 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 3, cache hit rate: 40.91%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-19 22:09:32] INFO:     127.0.0.1:34166 - "POST /v1/embeddings HTTP/1.1" 200 OK
Input IDs embedding (first 10): [0.0083160400390625, 0.0006837844848632812, -0.00809478759765625, -0.0006804466247558594, 0.0143585205078125, -0.0089874267578125, 0.0123748779296875, 0.002101898193359375, 0.006221771240234375, -0.003025054931640625]


In [6]:
terminate_process(embedding_process)